In [49]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Embedding, MaxPool1D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Bidirectional, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import glorot_uniform, RandomUniform, lecun_uniform, Constant
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from collections import OrderedDict
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPool1D
import tensorflow.keras.backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

Set the seeder to have as stable random operations as possible

In [2]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

Read data

In [3]:
bow_data = pd.read_csv('bow_data.csv') # bow
sequences_data = pd.read_csv('sequences_data.csv') # sequences of tokens

In [6]:
np.max(sequences_data["Length"])

392

In [7]:
word_counts = sequences_data["Vulnerability"].apply(lambda x: len(x.split()))
max_length = word_counts.max()
print("Maximum number of words:", max_length)


Maximum number of words: 392


In [8]:
print(sequences_data.head())

                                       Vulnerability       Category  Length
0              f"str$id""str$id""str$id"         ...  sql_injection       9
1      client.listentcp()    proxy = proxy(proxy_...           xsrf       8
2  from django.http import httpresponse, httpresp...  open_redirect       9
3  def write_preset(conn, queryin, descriptin):\t...  sql_injection     175
4                          update_query = self.up...  sql_injection      14


In [9]:
label_frequencies = sequences_data['Category'].value_counts()
print("Label Frequencies:\n", label_frequencies)
print("Total samples ", len(sequences_data))

Label Frequencies:
 sql_injection            1424
xsrf                      976
command_injection         721
path_disclosure           481
open_redirect             442
remote_code_execution     334
xss                       145
Name: Category, dtype: int64
Total samples  4523


Word Embedding

Word2Vec - load pre-trained word2vec embeddings - NL knowledge - static embeddings

In [10]:
import gensim.downloader
#w2v_vectors = gensim.downloader.load('word2vec-google-news-300')

In [11]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer models if not already downloaded
nltk.download('punkt')

tokenized_list = [word_tokenize(sentence) for sentence in sequences_data["Vulnerability"].tolist()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\iliaskaloup\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Encode each token using Word2Vec embeddings
def w2vEncoding(model, tokenized_list):
    encoded_list = []
    for sentence_tokens in tokenized_list:
        encoded_sentence = []
        for token in sentence_tokens:
            if token in model:
                encoded_token = model[token]
                encoded_sentence.append(encoded_token)
        encoded_list.append(encoded_sentence)
    
    return encoded_list

In [13]:
sequences_data["w2v"] = w2vEncoding(w2v_vectors, tokenized_list)

NameError: name 'w2v_vectors' is not defined

Use corpus to train word2vec vectors on python source code - PL knowledge - static embeddings

In [14]:
# # this should be executed only in the training set during cross-validation
# from gensim.models import Word2Vec

# w2v_model = Word2Vec(sentences=sequences_data["Vulnerability"], vector_size=300, window=5, min_count=1, workers=4)
# w2v_model.save("python_word2vec.model")
# #w2v_model = Word2Vec.load("python_word2vec.model")
# sequences_data["py_w2v"] = w2vEncoding(w2v_model.wv, tokenized_list)

BERT - load pre-trained bert embeddings - NL knowledge - contextual embeddings

In [15]:
model_variation = "bert-base-uncased" # "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_variation)
bert = TFAutoModel.from_pretrained(model_variation)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [16]:
bert_embeddings = bert.get_input_embeddings()
embedding_matrix = bert_embeddings.weights[0].numpy()
num_words = len(embedding_matrix)
print(num_words)
dim = len(embedding_matrix[0])
print(dim)

30522
768


In [17]:
sequences = [tokenizer.encode(sente, padding=True, truncation=True, add_special_tokens=False, return_tensors="tf").numpy() for sente in sequences_data["Vulnerability"]] # Tokenize the complete sentences

lines_pad = []
for seq in sequences:
    lines_pad.append(seq[0])

In [18]:
sequences_data["bert"] = lines_pad

CodeBERT - load pre-trained codebert embeddings - PL knowledge - contextual embeddings

In [19]:
model_variation = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=True)
codebert = TFAutoModel.from_pretrained(model_variation)

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at microsoft/codebert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [20]:
codebert_embeddings = codebert.get_input_embeddings()
embedding_matrix = codebert_embeddings.weights[0].numpy()
num_words = len(embedding_matrix)
print(num_words)
dim = len(embedding_matrix[0])
print(dim)

50265
768


In [21]:
sequences = [tokenizer(sente, return_tensors="tf", truncation=True, add_special_tokens=False) for sente in sequences_data["Vulnerability"]]


In [36]:
def padSequences(sequences, max_len):
    lines_pad = []
    for sequence in sequences:
        seq = sequence['input_ids'].numpy()[0]
        if len(seq) < max_len:
            for i in range(len(seq), max_len):
                seq = np.append(seq, 0)
        lines_pad.append(seq)
    return lines_pad

lines_pad = padSequences(sequences, 512)
lines_pad = [arr.tolist() for arr in lines_pad]
lines_pad = np.array(lines_pad)

In [37]:
sequences_data["codebert"] = lines_pad

In [38]:
sequences_data.iloc[0,:]

Vulnerability                f"str$id""str$id""str$id"         ...
Category                                             sql_injection
Length                                                           9
bert             [1042, 1000, 2358, 2099, 1002, 8909, 1000, 100...
codebert                                                      1437
Name: 0, dtype: object

RNN model, LSTM specifically

In [40]:
def buildLstm(max_len, top_words, dim, seed, embedding_matrix, multi):
    model=Sequential()
    #model.add(Embedding(input_dim=top_words+1, output_dim=dim, input_length=None, mask_zero=True))
    model.add(Embedding(input_dim=top_words, output_dim=dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    #model.add(SimpleRNN(300, dropout=0.3, stateful=False))
    model.add(LSTM(100, dropout=0.2, return_sequences=True, stateful=False))
    model.add(LSTM(50, dropout=0.1, stateful=False))
    #model.add(Bidirectional(LSTM(300, dropout=0.3, stateful=False)))
    #model.add(GRU(300, dropout=0.3, stateful=False))
    model.add(Activation('relu')) #dropout=0.2, recurrent_dropout=0.2, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)
    model.add(BatchNormalization(momentum=0.0))
    if multi == False:
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam')  
    else: 
        pass
    return model

CNN model 1-d

In [51]:
def buildCnn(max_len, top_words, dim, seed, embedding_matrix, multi):
    cnn_model = Sequential()
    cnn_model.add(Embedding(top_words, dim, input_length=None, weights=[embedding_matrix], mask_zero=True, trainable=False))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    '''cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))
    cnn_model.add(MaxPooling1D(pool_size = 5))
    cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = 'relu'))'''
    cnn_model.add(GlobalMaxPool1D())
    #cnn_model.add(Dense(units = 128, activation = 'relu'))
    if multi == False:
        cnn_model.add(Dense(units = 1, activation = 'sigmoid'))
        cnn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [recall_metric])
    else:
        pass
    return cnn_model

Cross-Validation

Binary Classification: Recognition of Injection Vulnerabilities (command_injection and sql_injection merged)

In [52]:
# Define functions for binary evaluation
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f1 = 2*((prec*rec)/(prec+rec+K.epsilon()))
    return f1

def f2_metric(y_true, y_pred):

    prec = precision_metric(y_true, y_pred)
    rec = recall_metric(y_true, y_pred)
    f2 = 5*((prec*rec)/(4*prec+rec+K.epsilon()))
    return f2

In [61]:
# Define a function to determine if the category is an injection or not
def is_injection(category):
    if category in ['sql_injection', 'command_injection']:
        return 1
    else:
        return 0

sequences_data['Injection'] = sequences_data['Category'].apply(is_injection)

In [62]:
def runCrossVal(X, y, max_len, num_words, dim, seed, embedding_matrix, userModel):
    ############## cross validation
    scores=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'f2']
    values = [np.array([]) for i in range(0, len(scores))]
    score_dict = OrderedDict(zip(scores, values))
    k=10
    f=0
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
    
    nb_epoch = 100
    BS = 64
    print("Training...")
    milli_sec1 = int(round(time.time() * 1000))
    
    for train_index, test_index in kfold.split(X, y):
        f = f + 1
        print('fold number= ',f)
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = y[train_index], y[test_index]
#         Y_train = np.array(Y_train)
#         Y_train = Y_train.ravel()
#         Y_test = np.array(Y_test)
#         Y_test = Y_test.ravel()
        
#         #sampling
#         X_res, Y_res = RandomOverSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
#         #X_res, Y_res = RandomUnderSampler(random_state=seed, sampling_strategy=0.5).fit_resample(X_train, Y_train)
    
#         #shuffle dataset
#         X_resampled=pd.DataFrame(X_res)
#         Y_resampled=pd.DataFrame(Y_res)
#         newTrain=X_resampled.assign(Label=Y_resampled.values)
#         newTrain = shuffle(newTrain,random_state=seed)
#         X_train=np.array(newTrain.iloc[:, 0:-1 ])
#         X_train=pd.DataFrame(X_train)
#         Y_train=np.array(newTrain.iloc[:, -1 ])
#         Y_train=pd.DataFrame(Y_train)
        
        if userModel == "cnn":
            myModel = buildCnn(max_len, num_words, dim, seed, embedding_matrix, False) 
        elif userModel == "lstm":
            myModel = buildLstm(max_len, num_words, dim, seed, embedding_matrix, False)
        print("model summary\m",myModel.summary())
        csv_logger = CSVLogger('log.csv', append=True, separator=',')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        history = myModel.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = nb_epoch, batch_size = BS, shuffle=False, verbose=1, callbacks=[csv_logger,es,mc])
        
        #load best model
        #model = load_model('best_model.h5')
        myModel.load_weights("best_model.h5")
        
        scores = myModel.evaluate(X_test, Y_test, verbose=0)
        #predictions = myModel.predict_classes(X_test, verbose=0)
        predictions = (myModel.predict(X_test) > 0.5).astype("int32")
        predScores = myModel.predict(X_test)
        accuracy=accuracy_score(Y_test, predictions)
        precision=precision_score(Y_test, predictions)
        recall=recall_score(Y_test, predictions)
        f1=f1_score(Y_test, predictions)
        roc_auc=roc_auc_score(Y_test, predictions)
        f2=5*precision*recall / (4*precision+recall)
        print(confusion_matrix(Y_test, predictions, labels=[0, 1]))
        tn, fp, fn, tp = confusion_matrix(Y_test, predictions).ravel()
        acc = ((tp+tn)/(tp+tn+fp+fn))
        print("Accuracy:%.2f%%"%(acc*100))
        print("Precision:%.2f%%"%(precision*100))
        print("Recall:%.2f%%"%(recall*100))
        print("F1 score:%.2f%%"%(f1*100))
        print("Roc_Auc score:%.2f%%"%(roc_auc*100))
        print("F2 score:%.2f%%"%(f2*100))
        print(classification_report(Y_test, predictions))
        del myModel
        score_dict['accuracy'] = np.append(score_dict['accuracy'], accuracy)
        score_dict['precision'] = np.append(score_dict['precision'], precision)
        score_dict['recall'] = np.append(score_dict['recall'], recall)
        score_dict['f1'] = np.append(score_dict['f1'], f1)
        score_dict['roc_auc'] = np.append(score_dict['roc_auc'], roc_auc)
        score_dict['f2'] = np.append(score_dict['f2'], f2)
        
    milli_sec2 = int(round(time.time() * 1000))
    print("Cross Validation is completed after", milli_sec2-milli_sec1)
    
    print("accuracy: %.2f%% (%.2f%%)" % (score_dict['accuracy'].mean()*100, score_dict['accuracy'].std()*100))
    print("precision: %.2f%% (%.2f%%)" % (score_dict['precision'].mean()*100, score_dict['precision'].std()*100))
    print("recall: %.2f%% (%.2f%%)" % (score_dict['recall'].mean()*100, score_dict['recall'].std()*100))
    print("f1: %.2f%% (%.2f%%)" % (score_dict['f1'].mean()*100, score_dict['f1'].std()*100))
    print("roc_auc: %.2f%% (%.2f%%)" % (score_dict['roc_auc'].mean()*100, score_dict['roc_auc'].std()*100))
    print("f2: %.2f%% (%.2f%%)" % (score_dict['f2'].mean()*100, score_dict['f2'].std()*100))

In [63]:
runCrossVal(sequences_data["codebert"].values, sequences_data["Injection"].values, 512, num_words, dim, seed, embedding_matrix, "lstm")

Training...
fold number=  1
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 768)         38603520  
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 100)         347600    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                30200     
_________________________________________________________________
activation_2 (Activation)    (None, 50)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50)                200       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 38,981,571
Trainable params: 377,951
Non-trainable params: 38,603,620
________

KeyboardInterrupt: 

Multi-class Classification: Categorization of all detected vulnerabilities

In [38]:
# Define functions for multiclass evalution

Use ML models and BoW code representation

Use DL modles and sequences of tokens code representation